In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

# Building an ETL Pipeline

### Setup your NYC Open Data variables ( app token change needed) 

In [ ]:
data_url = 'data.cityofnewyork.us'
data_set = 'ipu4-2q9a'
app_token = ''

In [ ]:
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

nyc open data client name is: <sodapy.socrata.Socrata object at 0x0000027CABEC0160>
nyc open data client data type is: <class 'sodapy.socrata.Socrata'>


### Step 2: Setup your Google BigQuery variables (Path & dataset_id change needed)

In [ ]:
key_path =  r''

In [ ]:
# run this cell without changing anything to setup your credentials
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x0000027CAFFD4A30>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [ ]:
dataset_id = ''   # PASTE THIS DATASET ID FROM ABOVE STEPS

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

your dataset_id is: cis-9440-361019.project


### Step 3: Extract data

#### The fist dataset (DOB Permit Issuance)

In [ ]:
# Get the total number of records in our the entire data set
total_record_count = nyc_open_data_client.get(data_set, select = "COUNT(*)")
print(f"total records in {data_set}: {total_record_count[0]['COUNT']}")

total records in ipu4-2q9a: 3921962


In [ ]:
target_record_count = nyc_open_data_client.get(data_set,
                                              where = "permit_type='NB'",
                                              select= "COUNT(*)")


print(f"target records in {data_set}: {int(target_record_count[0]['COUNT'])}")

target records in ipu4-2q9a: 258526


In [ ]:
# Now, loop through target data set to pull all rows in chunks (we cannot pull all rows at once)
# AGAIN, UPDATE WHERE FILTER INSIDE BELOW FUNCTION

def extract_socrata_data(chunk_size = 2500,
                         data_set = data_set,
                         where =None):
    
    # measure time this function takes
    import time
    start_time = time.time()
    
    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])
    
    # start at 0, empty list for results
    start = 0                   
    results = []                

    while True:

        if where == None:
            # fetch the set of records starting at 'start'
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))
            
        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size

        # if we have fetched all of the records (we have reached total_records), exit loop
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [ ]:
data = extract_socrata_data(chunk_size = 2500,
                            data_set = data_set,
                            where = "permit_type='NB'")

function took 76.8 seconds
the shape of your dataframe is: (258526, 58)


In [ ]:
data.columns

Index(['borough', 'bin__', 'house__', 'street_name', 'job__', 'job_doc___',
       'job_type', 'self_cert', 'block', 'lot', 'community_board', 'zip_code',
       'bldg_type', 'residential', 'permit_status', 'filing_status',
       'permit_type', 'permit_sequence__', 'site_fill', 'filing_date',
       'issuance_date', 'expiration_date', 'job_start_date',
       'permittee_s_first_name', 'permittee_s_last_name',
       'permittee_s_business_name', 'permittee_s_phone__',
       'permittee_s_license_type', 'permittee_s_license__',
       'superintendent_first___last_name', 'superintendent_business_name',
       'owner_s_business_type', 'non_profit', 'owner_s_business_name',
       'owner_s_first_name', 'owner_s_last_name', 'owner_s_phone__',
       'dobrundate', 'permit_si_no', 'gis_latitude', 'gis_longitude',
       'gis_council_district', 'gis_census_tract', 'gis_nta_name',
       'site_safety_mgr_s_first_name', 'site_safety_mgr_s_last_name',
       'site_safety_mgr_business_name', 'spec

In [ ]:
data = data[["borough","street_name","zip_code","bldg_type","permit_type","permit_status","permittee_s_business_name","filing_date","gis_latitude","gis_longitude"]]

In [ ]:
data

,borough,street_name,zip_code,bldg_type,permit_type,permit_status,permittee_s_business_name,filing_date,gis_latitude,gis_longitude
0,BROOKLYN,HERKIMER STREET,11213,2,NB,ISSUED,AB PLUS R CONSTRUCTION IN,05/10/2022,40.678919,-73.936482
1,BRONX,WHITE PLAINS ROAD,10470,2,NB,ISSUED,"SKF GENERAL CONTRACTORS,",05/10/2022,40.898971,-73.853874
2,BROOKLYN,WILLOUGHBY AVENUE,11205,2,NB,ISSUED,MC GOWAN BUILDERS INC,05/10/2022,40.692368,-73.961350
3,BROOKLYN,NEW YORK AVENUE,11203,2,NB,ISSUED,"M & Y DEVELOPERS, INC",05/10/2022,40.641580,-73.945684
4,BRONX,VYSE AVE,10460,2,NB,ISSUED,KIRI CONSTRUCTION CORP,05/10/2022,40.842322,-73.881900
...,...,...,...,...,...,...,...,...,...,...
258521,QUEENS,VILLAGE AVENUE,11691,2,NB,ISSUED,LETTIRE CONSTRUCTION CORP,03/15/2022,NaN,NaN
258522,QUEENS,VILLAGE LANE,11691,2,NB,ISSUED,LETTIRE CONSTRUCTION CORP,03/15/2022,NaN,NaN
258523,QUEENS,VILLAGE AVENUE,11691,2,NB,ISSUED,LETTIRE CONSTRUCTION CORP,03/29/2022,NaN,NaN
258524,QUEENS,VILLAGE LANE,11691,2,NB,ISSUED,LETTIRE CONSTRUCTION CORP,03/29/2022,NaN,NaN


In [ ]:
data['filing_date']= pd.to_datetime(data['filing_date'])

In [ ]:
data=data[(data['filing_date'] >= '2019-01-01') & (data['filing_date'] <= '2021-12-31')]

In [ ]:
data.isnull().sum()

borough                        0
street_name                    0
zip_code                      43
bldg_type                      0
permit_type                    0
permit_status                  9
permittee_s_business_name     81
filing_date                    0
gis_latitude                 440
gis_longitude                440
dtype: int64

In [ ]:
data=data.dropna()

In [ ]:
data.isnull().sum()

borough                      0
street_name                  0
zip_code                     0
bldg_type                    0
permit_type                  0
permit_status                0
permittee_s_business_name    0
filing_date                  0
gis_latitude                 0
gis_longitude                0
dtype: int64

In [ ]:
data = data.drop_duplicates()

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
data['year'] = pd.DatetimeIndex(data['filing_date']).year

In [ ]:
data['zip_code'] = data['zip_code'].astype('int64')
data['gis_latitude'] = data['gis_latitude'].astype('float')
data['gis_longitude'] = data['gis_longitude'].astype('float')

In [ ]:
data

,borough,street_name,zip_code,bldg_type,permit_type,permit_status,permittee_s_business_name,filing_date,gis_latitude,gis_longitude,year
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019
1,MANHATTAN,WEST 105TH STREET,10025,1,NB,ISSUED,TOP ROCK DEVELOPMENT AND,2021-11-17,40.798286,-73.962716,2021
2,BROOKLYN,61 STREET,11219,1,NB,ISSUED,KAUFMAN GROUP INC,2020-06-24,40.626170,-73.995670,2020
3,BRONX,SAINT LAWRENCE AVENUE,10473,1,NB,ISSUED,RASHBI DEVELOPMENT INC,2021-04-06,40.813572,-73.863039,2021
4,BROOKLYN,BEACH 38 STREET,11224,1,NB,ISSUED,SAPOL CONSTRUCTION LLC,2021-04-20,40.573632,-74.004341,2021
...,...,...,...,...,...,...,...,...,...,...,...
19277,BROOKLYN,SOUTH 5 STREET,11211,2,NB,ISSUED,TRIBOROUGH CONSTRUCTION S,2019-05-07,40.707104,-73.951759,2019
19278,BROOKLYN,SOUTH 5 STREET,11211,2,NB,ISSUED,TRIBOROUGH CONSTRUCTION S,2019-05-30,40.707104,-73.951759,2019
19279,BROOKLYN,SOUTH 5 STREET,11211,2,NB,ISSUED,DLC DEVELOPMENT CORP,2020-05-08,40.707104,-73.951759,2020
19280,QUEENS,103 AVE,11419,1,NB,ISSUED,CUSTOMBUILT HOMES INC,2020-08-17,40.688931,-73.821593,2020


#### The second dataset from Annualized Sales Update

In [ ]:
data1 = pd.read_csv('Borough_Sales_History_Merged.csv')

In [ ]:
data1 = data1.drop_duplicates()

In [ ]:
data1.columns

Index(['Unnamed: 0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AS OF FINAL ROLL 18/19', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AS OF FINAL ROLL 18/19', 'ADDRESS', 'APARTMENT NUMBER',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [ ]:
data1.drop(['Unnamed: 0', 'NEIGHBORHOOD','BOROUGH', 'BUILDING CLASS CATEGORY','TAX CLASS AS OF FINAL ROLL 18/19','BLOCK','LOT','EASE-MENT','BUILDING CLASS AS OF FINAL ROLL 18/19','APARTMENT NUMBER','RESIDENTIAL UNITS','COMMERCIAL UNITS','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT',"TAX CLASS AT TIME OF SALE",'BUILDING CLASS AT TIME OF SALE','ADDRESS'], axis=1, inplace=True)

In [ ]:
data1=data1.dropna()

In [ ]:
data1['ZIP CODE'] = data1['ZIP CODE'].astype('int64')
data1['SALE PRICE'] = data1['SALE PRICE'].astype('int64')
data1['SALE DATE']= pd.to_datetime(data1['SALE DATE'])

In [ ]:
data1.rename(columns = {'ZIP CODE':'zip_code', 'SALE PRICE':'median_housing_price',
                              'SALE DATE':'sale_date'}, inplace = True)

In [ ]:
data1['year'] = pd.DatetimeIndex(data1['sale_date']).year
data1=data1[data1['median_housing_price']>100]
data1.head()

,zip_code,median_housing_price,sale_date,year
0,10009,3200000,2019-07-24,2019
3,10009,6300000,2019-04-30,2019
5,10009,1950000,2019-08-08,2019
6,10009,14000000,2019-01-09,2019
8,10009,600000,2019-12-12,2019


In [ ]:
data1.dtypes

zip_code                         int64
median_housing_price             int64
sale_date               datetime64[ns]
year                             int64
dtype: object

In [ ]:
data1 =data1.groupby(['zip_code', 'year']).median()
data1

median_housing_price
zip_code year                      
10001    2019             2938784.0
         2020             2762500.0
         2021             2045000.0
10002    2019             1419368.0
         2020             1083750.0
...                             ...
11694    2020              624391.0
         2021              722500.0
11697    2019              433500.0
         2020              500000.0
         2021              625000.0

[559 rows x 1 columns]

#### Merge two datasets together

In [ ]:
data= data.merge(data1, on=["zip_code","year"])

In [ ]:
data['new_permit_quantity']=1

In [ ]:
data.rename(columns = {'street_name':'street','permittee_s_business_name':'permittees_business_name',
                              'filing_date':'date','gis_latitude':'latitude','gis_longitude':'longitude'}, inplace = True)

In [ ]:
data['median_housing_price'] = data['median_housing_price'].astype('int64')

In [ ]:
data

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1
2,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1
3,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1
4,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-01-04,40.636955,-73.950995,2019,772000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19265,MANHATTAN,WEST 218 STREET,10034,2,NB,ISSUED,PLAZA CONSTRUCTION LLC,2021-10-21,40.872927,-73.918203,2021,465000,1
19266,MANHATTAN,EAST 74TH STREET,10021,2,NB,ISSUED,TURNER CONSTRUCTION COMPA,2021-10-28,40.767810,-73.951956,2021,1100000,1
19267,MANHATTAN,EAST 74TH STREET,10021,2,NB,RE-ISSUED,TURNER CONSTRUCTION COMPA,2021-10-28,40.767851,-73.952054,2021,1100000,1
19268,MANHATTAN,EAST 74TH STREET,10021,2,NB,ISSUED,TURNER CONSTRUCTION COMPA,2021-10-26,40.767851,-73.952054,2021,1100000,1


In [ ]:
data.to_csv("data.csv",index=False)

### Step 4: Data Profiling

In [ ]:
data.columns

Index(['borough', 'street', 'zip_code', 'bldg_type', 'permit_type',
       'permit_status', 'permittees_business_name', 'date', 'latitude',
       'longitude', 'year', 'median_housing_price', 'new_permit_quantity'],
      dtype='object')

In [ ]:
# create and run a function to ceate data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [ ]:
data_profiling_df = create_data_profiling_df(data=data)

C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_2460\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_2460\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_2460\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_2460\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pand

In [ ]:
data_profiling_df

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
9,longitude,float64,6985,12285,0,19270
8,latitude,float64,6969,12301,0,19270
1,street,object,3197,16073,0,19270
6,permittees_business_name,object,1801,17469,0,19270
7,date,datetime64[ns],891,18379,0,19270
11,median_housing_price,int64,378,18892,0,19270
2,zip_code,int64,178,19092,0,19270
0,borough,object,5,19265,0,19270
5,permit_status,object,3,19267,0,19270
10,year,int64,3,19267,0,19270


### Step 5: Data Cleansing

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19270 entries, 0 to 19269
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   borough                   19270 non-null  object        
 1   street                    19270 non-null  object        
 2   zip_code                  19270 non-null  int64         
 3   bldg_type                 19270 non-null  object        
 4   permit_type               19270 non-null  object        
 5   permit_status             19270 non-null  object        
 6   permittees_business_name  19270 non-null  object        
 7   date                      19270 non-null  datetime64[ns]
 8   latitude                  19270 non-null  float64       
 9   longitude                 19270 non-null  float64       
 10  year                      19270 non-null  int64         
 11  median_housing_price      19270 non-null  int64         
 12  new_permit_quantit

In [ ]:
# find number of duplicate rows

print(f"number of duplicate rows: {len(data[data.duplicated()])}")

number of duplicate rows: 0


In [ ]:
data = data.drop_duplicates(keep = 'first')
print(f"number of rows after duplicates dropped: {len(data)}")

number of rows after duplicates dropped: 19270


### Step 6: Create location dimension table

In [ ]:
# first, copy the entire table
Location_dim = data.copy()

In [ ]:
# second, subset for only the wanted columns in the dimension
Location_dim = Location_dim[['borough','zip_code','street','longitude','latitude']]
Location_dim.shape

(19270, 5)

In [ ]:
# third, drop duplicate rows in dimension
unique_row =['borough','zip_code','street','longitude','latitude']
Location_dim = Location_dim.drop_duplicates(subset = unique_row, keep = 'first')
Location_dim = Location_dim.reset_index(drop = True)
Location_dim.shape

(7238, 5)

In [ ]:
# fourth, add location_id as a surrogate key
Location_dim.insert(0, 'location_id', range(1, 1 + len(Location_dim)))

In [ ]:
Location_dim

,location_id,borough,zip_code,street,longitude,latitude
0,1,BROOKLYN,11210,EAST 22 STREET,-73.953736,40.622307
1,2,BROOKLYN,11210,FLATBUSH AVENUE,-73.953055,40.637919
2,3,BROOKLYN,11210,EAST 32 STREET,-73.945784,40.633264
3,4,BROOKLYN,11210,NEW YORK AVENUE,-73.945176,40.636835
4,5,BROOKLYN,11210,ROGERS AVENUE,-73.950995,40.636955
...,...,...,...,...,...,...
7233,7234,QUEENS,11427,85 AVENUE,-73.751341,40.731260
7234,7235,MANHATTAN,10038,BEEKMAN STREET,-74.006298,40.711050
7235,7236,MANHATTAN,10034,BROADWAY,-73.926075,40.865947
7236,7237,MANHATTAN,10034,WEST 218 STREET,-73.912431,40.870513


In [ ]:
# fifth, add the location_id to the df table
data = data.merge(Location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

In [ ]:
data.head()

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2
2,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,3
3,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,4
4,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-01-04,40.636955,-73.950995,2019,772000,1,5


### Step 7: Create date dimension table

In [ ]:
# first, copy the entire table
dateDim = data.copy()

In [ ]:
dateDim = dateDim[["date",'year']]

In [ ]:
dateDim=dateDim[~dateDim['date'].dt.date.duplicated()]
dateDim['month'] = dateDim['date'].dt.month
# create date_id column in the dateDim table
dateDim['date_id'] = dateDim['date'].apply(lambda x: pd.to_datetime(x).strftime("%Y%m%d"))
dateDim = dateDim.reset_index(drop = True)

In [ ]:
dateDim

,date,year,Month,date_id
0,2019-09-24,2019,9,20190924
1,2019-05-10,2019,5,20190510
2,2019-04-30,2019,4,20190430
3,2019-01-03,2019,1,20190103
4,2019-01-04,2019,1,20190104
...,...,...,...,...
886,2021-09-06,2021,9,20210906
887,2021-04-11,2021,4,20210411
888,2021-03-13,2021,3,20210313
889,2021-12-11,2021,12,20211211


In [ ]:
# create date_id column in the data table
data['date_id'] = data['date'].apply(lambda x: pd.to_datetime(x).strftime("%Y%m%d"))

In [ ]:
data

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id,date_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1,20190924
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2,20190510
2,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,3,20190430
3,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,4,20190103
4,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-01-04,40.636955,-73.950995,2019,772000,1,5,20190104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19265,MANHATTAN,WEST 218 STREET,10034,2,NB,ISSUED,PLAZA CONSTRUCTION LLC,2021-10-21,40.872927,-73.918203,2021,465000,1,7238,20211021
19266,MANHATTAN,EAST 74TH STREET,10021,2,NB,ISSUED,TURNER CONSTRUCTION COMPA,2021-10-28,40.767810,-73.951956,2021,1100000,1,6697,20211028
19267,MANHATTAN,EAST 74TH STREET,10021,2,NB,RE-ISSUED,TURNER CONSTRUCTION COMPA,2021-10-28,40.767851,-73.952054,2021,1100000,1,6696,20211028
19268,MANHATTAN,EAST 74TH STREET,10021,2,NB,ISSUED,TURNER CONSTRUCTION COMPA,2021-10-26,40.767851,-73.952054,2021,1100000,1,6696,20211026


### Step 8: Create permit dimension table

In [ ]:
# first, copy the entire table

Permit_dim = data.copy()

In [ ]:
# second, subset for only the wanted columns in the dimension

Permit_dim = Permit_dim[["permit_type","permittees_business_name","permit_status"]]

In [ ]:
# third, drop duplicate rows in dimension

unique_row = ["permit_type","permittees_business_name","permit_status"]
Permit_dim = Permit_dim.drop_duplicates(subset = unique_row, keep = 'first')
Permit_dim = Permit_dim.reset_index(drop = True)
Permit_dim

,permit_type,permittees_business_name,permit_status
0,NB,LOUIS PERL CONTRACTING LT,ISSUED
1,NB,SUPREME BUILDERS & DEVELO,ISSUED
2,NB,Y&L INTERIOR LLC,ISSUED
3,NB,LEMLE & WOLFF CONSTR CORP,ISSUED
4,NB,M. MELNICK & COMPANY INC,ISSUED
...,...,...,...
2129,NB,M.N.C. GENERAL CONTR CORP,IN PROCESS
2130,NB,M W MANAGEMENT & DEV CORP,RE-ISSUED
2131,NB,ZOITCO CONSTRUCTION CORP,ISSUED
2132,NB,K'S CONSTRUCTION NY INC,ISSUED


In [ ]:
# fourth, add permit_id as a surrogate key

Permit_dim.insert(0, 'permit_id', range(1000, 1000 + len(Permit_dim)))
Permit_dim

,permit_id,permit_type,permittees_business_name,permit_status
0,1000,NB,LOUIS PERL CONTRACTING LT,ISSUED
1,1001,NB,SUPREME BUILDERS & DEVELO,ISSUED
2,1002,NB,Y&L INTERIOR LLC,ISSUED
3,1003,NB,LEMLE & WOLFF CONSTR CORP,ISSUED
4,1004,NB,M. MELNICK & COMPANY INC,ISSUED
...,...,...,...,...
2129,3129,NB,M.N.C. GENERAL CONTR CORP,IN PROCESS
2130,3130,NB,M W MANAGEMENT & DEV CORP,RE-ISSUED
2131,3131,NB,ZOITCO CONSTRUCTION CORP,ISSUED
2132,3132,NB,K'S CONSTRUCTION NY INC,ISSUED


In [ ]:
#Merge
data = data.merge(Permit_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(10)

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id,date_id,permit_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1,20190924,1000
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2,20190510,1001
2,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,3,20190430,1002
3,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,4,20190103,1003
4,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-01-04,40.636955,-73.950995,2019,772000,1,5,20190104,1004
5,BROOKLYN,EAST 32ND ST,11210,2,NB,ISSUED,SUPERB PROPERTY MNG'T,2019-05-07,40.633299,-73.945770,2019,772000,1,6,20190507,1005
6,BROOKLYN,EAST 35TH STREET,11210,2,NB,ISSUED,SINGLA CONCEPTS INC,2019-03-14,40.633586,-73.942913,2019,772000,1,7,20190314,1006
7,BROOKLYN,EAST 31 STREET,11210,1,NB,ISSUED,RELIANT BUILDERS CORP,2019-05-15,40.623316,-73.945209,2019,772000,1,8,20190515,1007
8,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,BIG DREAM DEVELOPERS LLC,2019-05-15,40.634770,-73.944933,2019,772000,1,9,20190515,1008
9,BROOKLYN,EAST 32ND ST,11210,2,NB,ISSUED,SUPERB PROPERTY MNG'T,2019-05-07,40.632564,-73.945691,2019,772000,1,10,20190507,1005


### Step 9: Creating Fact Table

In [ ]:
# take a subset of fact_table for only the needed columns: which are keys and measures

New_building_permits_facts = data[['permit_id','date_id','location_id','median_housing_price','new_permit_quantity']]
New_building_permits_facts

,permit_id,date_id,location_id,median_housing_price,new_permit_quantity
0,1000,20190924,1,772000,1
1,1001,20190510,2,772000,1
2,1002,20190430,3,772000,1
3,1003,20190103,4,772000,1
4,1004,20190104,5,772000,1
...,...,...,...,...,...
19265,1363,20211021,7238,465000,1
19266,1924,20211028,6697,1100000,1
19267,3013,20211028,6696,1100000,1
19268,1924,20211026,6696,1100000,1


### Step 10: Load to BigQuery

In [ ]:
New_building_permits_facts.to_csv("New_building_permits_facts.csv",index=False)

In [ ]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id #change 301800 to match your project id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"completed job {load_job}")

In [ ]:
#Sending Permit_dim table

load_table_to_bigquery(df = Permit_dim,
                       table_name = "Permit_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=e5369e37-7838-453f-876c-186ca1db65e8>


In [ ]:
#Sending Date_dim table

load_table_to_bigquery(df = dateDim,
                       table_name = "Date_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=db094f92-b0b0-4af7-ad16-eb9b29927a01>


In [ ]:
#Sending Location_dim table

load_table_to_bigquery(df = Location_dim,
                       table_name = "Location_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=775c1396-a8f1-4284-b83f-502e346ae1e1>


In [ ]:
#Sending Fact table

load_table_to_bigquery(df = New_building_permits_facts,
                       table_name = "New_building_permits_facts",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=01f72739-6393-4b1a-9bf3-c64e9dbd85de>
